### Update information: Input required
#### Note: quotation marks should be included. Change values inside of the quotes.

In [1]:
AY = '2022-2023'
CO = 'c/o 2024'
Clerks = 'Surgery'

That's it. Now you can sit back and run the code! 
### Clerkship Data Automation

In [2]:
# initiate
import numpy as np
import pandas as pd
import re
from datetime import datetime
import datetime
import time
import warnings
warnings.simplefilter("ignore")

# start timer
tic = time.time()

# import dataset
readfile = pd.read_excel('input.xlsx', engine='openpyxl')

# subset site survey questions
q_subset = readfile.loc[:,readfile.columns.str.contains('.*Q2.*')]

# create a list of columns
q_cols_subset = list(q_subset.columns)

# subset overall survey questions
q_subset_o = readfile.loc[:,readfile.columns.str.contains('Q3.*')]

# create a list of overall columns
q_cols_subset_o = list(q_subset_o.columns)

# calculate the number of one set of questions
set_n = sum(1 for i in q_cols_subset if i[0:2] == (q_cols_subset[0])[0:2])

# create a dataframe for stats
df1 = pd.DataFrame([])
qn=0
for j in q_cols_subset:

    c = q_subset[j][1:,].value_counts().rename_axis('unique_values').reset_index(name='counts')

    
    if (qn == set_n):
        qn = 1

    else:
        qn +=1
        
    c['QN'] = qn

    cc = {}
    
    for i in range(1,6):
        if i in list(c.unique_values):
            cn = c[c.unique_values == i]['counts']

            cc.update(                {
                    'C'+str(i):int(cn)
                }            )
            
        #filter out invalid values (not a number between 1-5: Text or other coded value for N/A)
        else:
            cc.update(                {
                    'C'+str(i):0
                }              )


    dd = pd.DataFrame([cc])
    dd['N'] = dd.C1 + dd.C2 + dd.C3 + dd.C4 + dd.C5
    dd['Sumproduct'] = dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5
    dd['Mean'] = (dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5)/dd.N
    dd['QN']= c['QN']
    df1 = pd.concat([df1, dd], axis=0, ignore_index=True)
    df1 = df1[['QN','Mean', 'N','Sumproduct']]
#df1

# create a dataframe for question by extracting text
sequence2=0
q0 = []
for l in q_cols_subset:
    q_text_q = str(q_subset[l][0])  
    q0.append(q_text_q)


df_q1 = pd.DataFrame(q0)
df_q1.columns = ['Question']
#df_q1

# create a dataframe for site
sequence1 = 0
site = []

# extract from the first question & repeat
for k in q_cols_subset:
    sequence1 = sequence1 + 1
    if (sequence1%set_n == 1):            
        q_text_f = str(q_subset[k][0])
        res = re.findall(r'\-\s(.+)\s\-', q_text_f)

        # result
        site.append(str(res[0]))


    else:
        site.append(str(res[0]))
        continue

df_s = pd.DataFrame(site,
       columns =['Site'])
#df_s

# create overall stats
df2 = df1.groupby(['QN'], as_index=False).sum()
df2['Mean'] = df2['Sumproduct'] / df2['N']
df2['Question']=df2['QN']
df2 = df2[['Question','Mean', 'N']]
df2['Site'] = 'Overall'
#df2

# create a dataframe for stats
df3 = pd.DataFrame([])
qn=0
for j in q_cols_subset_o:

    c = q_subset_o[j][1:,].value_counts().rename_axis('unique_values').reset_index(name='counts')

    
    if (qn == set_n):
        qn = 1

    else:
        qn +=1
        
    c['QN'] = qn

    cc = {}
    
    for i in range(1,6):
        if i in list(c.unique_values):
            cn = c[c.unique_values == i]['counts']

            cc.update(                {
                    'C'+str(i):int(cn)
                }            )
            
        #filter out invalid values (not a number between 1-5: Text or other coded value for N/A)
        else:
            cc.update(                {
                    'C'+str(i):0
                }              )


    dd = pd.DataFrame([cc])
    dd['N'] = dd.C1 + dd.C2 + dd.C3 + dd.C4 + dd.C5
    dd['Sumproduct'] = dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5
    dd['Mean'] = (dd.C1 * 1 + dd.C2 * 2 + dd.C3 * 3 + dd.C4 * 4 + dd.C5 * 5)/dd.N
    dd['QN']= c['QN']
    df3 = pd.concat([df3, dd], axis=0, ignore_index=True)
    df3 = df3[['QN','Mean', 'N','Sumproduct']]
#df3

# create a dataframe for question by extracting text
sequence2=0
q0 = []
for l in q_cols_subset_o:
    q_text_q = str(q_subset_o[l][0])  
    q0.append(q_text_q)


df_q2 = pd.DataFrame(q0)
df_q2.columns = ['Question']
#df_q2

# concat dataframes
df_s1 = pd.concat([df_s, df_q1, df1[['Mean','N']]], axis=1)
df_c1 = pd.concat([df_s1, df2], axis=0)
df_o1 = pd.concat([df_q2, df3[['Mean','N']]], axis=1)
df_o1['Site']='Overall'
df4 = pd.concat([df_c1, df_o1], axis=0)
#df4

# delete unapplicable rows/qualitative questions
df5 = df4[df4['N'] > 0]

# populate columns for AY, Class, Block
df5['AcademicYear'] = AY
df5['Class'] = CO
df5['Clerkship'] = Clerks
df5['Type'] = 'Mean'
df5['Value'] = ''
df5 = df5[['AcademicYear', 'Clerkship', 'Site', 'Question', 'Type', 'Value', 'Mean', 'N', 'Class']]
#df5

# export
df5.to_excel('output_' + str(datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")) + '.xlsx', index = False)

# end timer             
toc = time.time() - tic
print('All tasks completed in %.2f s.' % (toc))

All tasks completed in 2.40 s.
